In [1]:
# import cf, cfplot as cfp
import numpy as np
import scipy as sp
import sys
import os
from netCDF4 import Dataset
import netCDF4 as nc
%matplotlib inline
import matplotlib
import xarray as xr
import matplotlib.pyplot as plt
import warnings

import matplotlib.pyplot as plt
import matplotlib.path as mpath
# Quick plot to show the results
from cartopy import config
import cartopy.crs as ccrs
import cartopy
import cartopy.feature as cfeature
import scipy.ndimage as ndimage
# from mpl_toolkits.basemap import Basemap
# from mpl_toolkits.basemap import addcyclic
from cartopy.util import add_cyclic_point
import matplotlib.pylab as pl
from scipy import stats
import regionmask
import glob
import dask
dask.config.set(**{'array.slicing.split_large_chunks': True})
print(xr.__version__)

0.20.2


Calculate Jet latitude following Ceppi et al. 2018

In [4]:
idir='/gws/nopw/j04/realproj/users/adittus'
def rebase(dataset,files):
    # ds = xr.open_dataset('intermediate_files/stable/gmst/tas_gmst_bq777.nc') #,engine='zarr')
    dates=dataset[0,].dropna(dim='year',how='all').year  
    datasets = []
    test=dataset
    for i in np.arange(0,int(dataset.shape[0])):
        tmp=test[i,:].dropna(dim='year',how='all')
        # print(tmp)
        # print(tmp.shape)
        # print(dates)
        tmp['year']=dates
#         print(tmp.year)
        datasets.append(tmp)
    dsnew = xr.concat(datasets, dim='cases')
    # print(dsnew)
    return(dsnew)
def preprocess(ds):
    print(ds.encoding['source'])
    return(ds)
def seasaverage_3(var,seas):
    season={'DJF':0,'JFM':1,'FMA':2,'MAM':3,'AMJ':4,'MJJ':5,'JJA':6,'JAS':7,'ASO':8,'SON':9,'OND':10,'NDJ':11,'ann':None}
    seasindex=season[seas]
    if (seas == 'ann'):
        var_seas=var.groupby('time.year').mean('time')
    else:
        if (var.ndim == 4):
            var_seas=var.rolling(min_periods=3, center=True, time=3).mean()[:,seasindex::12,:,:]
            var_seas=var_seas.groupby('time.year').mean('time') #rename({"time":"year"})
        elif (var.ndim == 3): 
            var_seas=var.rolling(min_periods=3, center=True, time=3).mean()[seasindex::12,:,:]
            var_seas=var_seas.groupby('time.year').mean('time')
        print(var_seas)
    return(var_seas)
zipgmststab=sorted(glob.glob('/gws/nopw/j04/realproj/users/adittus/intermediate_files/stable/gmst/tas_gmst_*.nc'))
zipgmst=xr.open_mfdataset(zipgmststab,combine='nested',concat_dim='cases',preprocess=None)['tas']
zipgmst=np.nanmean(rebase(zipgmst,zipgmststab),axis=1)
# print(zipgmst)
def sortbygmst(input):
    input = [x for _,x in sorted(zip(zipgmst,input))]
    return(input)

In [ ]:
seas = 'JJA'

def lonflip(var):
    var.coords['lon'] = xr.where(var.coords['lon'] > 180, var.coords['lon'] - 360, var.coords['lon'])
    return var.sortby("lon", ascending=True)

jet_lat = {}
jet_lat_transient = {}
jet_lat_clim = {}
jet_lat_annual = {}
jet_lat_annual_transient = {}
jet_lat_annual_hist = {}
jet_lat_hist = {}

for region in ['Atlantic']:
    print(region)
    stablefiles = sortbygmst(sorted(glob.glob(idir + '/intermediate_files/stable/ua_850/ua_850*nc')))
    
    ua = seasaverage_3(xr.open_mfdataset(stablefiles, combine='nested', concat_dim='cases', preprocess=preprocess)['ua'], seas)
    gmst_base = xr.open_mfdataset(idir + '/intermediate_files/historical/gmst/tas_gmst*nc', combine='nested', concat_dim='cases', preprocess=preprocess)['tas'][:, 0:50].mean('year').mean('cases')
    gmst = xr.open_mfdataset(idir + '/intermediate_files/stable/gmst/tas_gmst*nc', combine='nested', concat_dim='cases', preprocess=preprocess)['tas'] - gmst_base
    ua_base = seasaverage_3(xr.open_mfdataset(idir + '/intermediate_files/historical/ua_850/ua_850*nc', combine='nested', concat_dim='cases', preprocess=preprocess)['ua'], seas)[:, 0:50, :, :].mean('year').mean('cases')
    ua_transient = seasaverage_3(xr.open_mfdataset(idir + '/intermediate_files/ssp370/ua_850/ua_850*nc', combine='nested', concat_dim='cases', preprocess=preprocess)['ua'], seas)
    ua_hist = seasaverage_3(xr.open_mfdataset(idir + '/intermediate_files/historical/ua_850/ua_850*nc', combine='nested', concat_dim='cases', preprocess=preprocess)['ua'], seas)
    gmst_transient = xr.open_mfdataset(idir + '/intermediate_files/ssp370/gmst/tas_gmst*nc', combine='nested', concat_dim='cases', preprocess=preprocess)['tas'] - gmst_base
    pic = ua_base  # xr.open_mfdataset(idir+'/intermediate_files/piControl/ua/ua*nc',combine='nested',concat_dim='time',preprocess=preprocess)['ua'].mean('time')

    if region == 'Atlantic':
        ua = lonflip(ua)
        ua_transient = lonflip(ua_transient)
        pic = lonflip(pic)
        ua_hist = lonflip(ua_hist)
        lonmask = (ua.lon >= -60) & (ua.lon <= 60)
    elif region == 'Pacific':
        lonmask = (ua.lon >= 140) & (ua.lon <= 240)

    zonal_ua = ua[:, :, :, lonmask].mean("lon")
    zonal_ua_transient = ua_transient[:, :, :, lonmask].mean("lon")
    zonal_ua_hist = ua_hist[:, :, :, lonmask].mean("lon")
    zonal_pic = pic[:, lonmask].mean("lon")

    delta_lat = 1.25
    latmask = (zonal_ua.lat >= 30.) & (zonal_ua.lat <= 60.) & (zonal_ua_transient.mean(("cases","year")) > 0.)

    masked_zonal_ua = zonal_ua[:, :, latmask]
    masked_zonal_ua_transient = zonal_ua_transient[:, :, latmask]
    masked_zonal_ua_hist = zonal_ua_hist[:, :, latmask]

    jet_lat[region] = ((masked_zonal_ua.lat * masked_zonal_ua**2).integrate(coord='lat')) / ((masked_zonal_ua**2).integrate(coord='lat'))
    jet_lat_transient[region] = (masked_zonal_ua_transient.lat * masked_zonal_ua_transient**2).integrate(coord='lat') / (masked_zonal_ua_transient**2).integrate(coord='lat')
    jet_lat_hist[region] = (masked_zonal_ua_hist.lat * masked_zonal_ua_hist**2).integrate(coord='lat') / (masked_zonal_ua_hist**2).integrate(coord='lat')
    
    masked_zonal_clim = zonal_pic[latmask]
    jet_lat_clim[region] = (masked_zonal_clim.lat * masked_zonal_clim**2).integrate(coord='lat') / (masked_zonal_clim**2).integrate(coord='lat')

    jet_lat_annual[region] = jet_lat[region]
    jet_lat_annual_transient[region] = jet_lat_transient[region]
    jet_lat_annual_hist[region] = jet_lat_hist[region]
    print(jet_lat_clim[region].values)

netcdf_dataset=xr.Dataset(
     data_vars=dict(
    jet_lat_Atlantic_stable=(["runs_real", "year_1"], jet_lat_annual['Atlantic'].values),
    jet_lat_Atlantic_transient=(["runs_cmip6", "year_2"], jet_lat_annual_transient['Atlantic'].values)
     ),
    # coords=dict(
    #     lon=(["x", "y"], lon),

    #     time=time,
    #     reference_time=reference_time,
    # ),
    attrs=dict(region=region,
              season=seas),
)
netcdf_dataset.to_netcdf('timeseries/jet_lat_'+seas+'.nc')



Atlantic
/gws/nopw/j04/realproj/users/adittus/intermediate_files/stable/ua_850/ua_850_bq777.nc
/gws/nopw/j04/realproj/users/adittus/intermediate_files/stable/ua_850/ua_850_bw848.nc
/gws/nopw/j04/realproj/users/adittus/intermediate_files/stable/ua_850/ua_850_bz227.nc
/gws/nopw/j04/realproj/users/adittus/intermediate_files/stable/ua_850/ua_850_cd269.nc
/gws/nopw/j04/realproj/users/adittus/intermediate_files/stable/ua_850/ua_850_bw987.nc
/gws/nopw/j04/realproj/users/adittus/intermediate_files/stable/ua_850/ua_850_bu607.nc
<xarray.DataArray 'ua' (cases: 6, year: 526, lat: 145, lon: 192)>
dask.array<transpose, shape=(6, 526, 145, 192), dtype=float64, chunksize=(1, 1, 145, 192), chunktype=numpy.ndarray>
Coordinates:
    plev     float64 8.5e+04
  * lat      (lat) float64 -90.0 -88.75 -87.5 -86.25 ... 86.25 87.5 88.75 90.0
  * lon      (lon) float64 0.0 1.875 3.75 5.625 7.5 ... 352.5 354.4 356.2 358.1
  * year     (year) int64 2014 2015 2016 2017 2018 ... 2535 2536 2537 2538 2539
Dimensions w